# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [17]:
import pandas as pd

df = pd.read_csv('/datasets/data.csv')

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Предварительный просмотр данных:
1. days_employed - есть отрицательные значения, необходимо будет изучить, есть ли закономерность между отрицательным значением days_employed и какого-либо показателя (для удобства можно будет перевести дни в годы (?) или в целочисленное значение)
2. education - данные об образовании записано в разном формате: например, при группировке среднее, Среднее и СРЕДНЕЕ будет считаться по отдельности.
3. Можно также total_income перевести в целочисленные значения - на статистику это повлияет вряд ли, а вот визуально будет смотреться лучше.

Общая информация о данных таблицы df:

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Вывод: Предврарительный вывод по имеющимся данным можно сделать такой:
1. days_employed - есть пропущенные значения, + т.к. это поле с количеством дней трудового стажа, так что формат данных должен быть - int
2. debt - int? (вообще формат д.б. boolean, но скорее всего это не понадобится)
4. total_income - есть пропущенные данные

Важные для ответы на вопросы данные:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок? children и debt (похоже, достаточно будет добавить флаг есть дети/нет детей)
2. Есть ли зависимость между семейным положением и возвратом кредита в срок? family_status и debt
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок? total_income и debt 
4. Как разные цели кредита влияют на его возврат в срок? purpose и debt

В первую очередь необходимо проверить корректность данных в этих 5 столбцах. 

In [19]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Вывод: с названиями столбцов все в порядке

In [20]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как видим, в days_employed и total_income действительно есть пропущенные значения (проверить, с чем это может быть связано)

**Вывод:** Посмотрели на данные, определили формат данных, какие есть слабые стороны этих данных, что надо исправить и на что лучше обратить внимание.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [21]:
df[df.isna().any(axis=1)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Какой-то очевидной зависимости наличия пропущенных значений нет.
Т.к. зависимость дней трудового стажа и возврата средств нас не интересует, то пропущенные данные в 
days_employed можно заменить на медиану. Для начала необходимо исправить отрицательные значения, т.к. они повлияют на медиану, посмотрим, сколько их:

In [22]:
df[df['days_employed'] < 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [23]:
negative_days = len(df[df['days_employed'] < 0]) / len(df) 
print('Отрицательный стаж у {:.0%}'.format(negative_days))

Отрицательный стаж у 74%


74% строк с отрицательными значениями не выглядит как случайность. Похоже, что-то не так со сбором данных.  
Но т.к. зависимость трудового стажа и возвратом кредита нас сейчас не интересует, заменим отрицательные значения на положительные, а пропущенные - на медиану. 

In [24]:
df['days_employed'] = df['days_employed'].abs()

Проверяем, не осталось ли строк с отрицательным трудовым стажем:

In [25]:
len(df[df['days_employed'] < 0])

0

Отлично! Теперь можно считать медиану по типу занятости:

In [10]:
# median = df['days_employed'].median()
# print(median)

# median_day_employed_by_income_type = df.groupby('income_type').agg({'days_employed' : 'median'})
# print(median_day_employed_by_income_type)

Заменим пропущенные значения на медиану в зависимости от типа занятости:

In [26]:
df['days_employed'] = df.groupby('income_type')['days_employed'].apply(lambda x: x.fillna(x.median()))
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


340 тысяч дней - неплохой такой стаж! Посмотрим, сколько это лет:

In [27]:
print(340266 / 365)

932.2356164383561


Вот это да! Давайте посчитаем, сколько таких трудоголиков и кто они:

In [28]:
len(df[df['days_employed'] > 300000])
print(len(df[df['days_employed'] > 300000]), 'человек работали больше 900 лет')

3858 человек работали больше 900 лет


In [29]:
df[df['days_employed'] > 300000].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,365213.306266,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости


In [30]:
df[df['days_employed'] > 300000]['income_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

И такой стаж только у пенсионеров и безработных! Что-то здесь не то. Может, тут считались не дни, а часы?

In [31]:
print(round(340266 / 24/ 365))

39


Уже похоже на правду. Давайте все-таки заменим эти аномальные значения на более адекватные (переведем часы в дни):

In [33]:
df['days_employed'] = df['days_employed'].apply(lambda x: x/24 if x > 300000 else x)
# print(df[df['days_employed'] > 300000]['days_employed'])

Проверим, сработало ли: 

In [36]:
df[df['days_employed'] > 300000].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Чудно! Теперь все у всех хорошо с трудовым стажем:

In [19]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


15 тысяч дней тоже как-то особенно выделяются, надо проверить, все ли в порядке с данными:

In [20]:
df['days_employed'].describe()

count    21525.000000
mean      4600.535795
std       5354.354167
min         24.141633
25%       1025.549623
50%       1993.522017
75%       5347.024506
max      18388.949901
Name: days_employed, dtype: float64

Так, теперь максимальный стаж - 18 тысяч дней. Выглядит внушительно по сравнению с остальными цифрами. Проверим на всякий случай - посчитаем сколько это лет:

In [37]:
print(18388.949901/365)

50.38068466027397


А, ну всё в порядке! Люди могут работать так много дней. 

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [39]:
df['days_employed'].isna().sum()

0

Отлично! Заменили пропущенные значения на медиану по типу занятости, а аномально большие значения пересчитали. И тоже заменили. Идем дальше.

Теперь необходимо детально рассмотреть данные в total_income - вот они нам точно понадобятся. 

In [40]:
df['total_income'].isna().sum()

2174

In [41]:
df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,15217.221094,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,2689.368353,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,15217.221094,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,2689.368353,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,15217.221094,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,1547.382223,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,1574.202821,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,1547.382223,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,1574.202821,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [42]:
print('Максимальная зарплата', df['total_income'].max())
print('Минимальная запрлата', df['total_income'].min())
print('Средняя зарплата', df['total_income'].mean())
print('Медианная зарплата', df['total_income'].median())
median = df['total_income'].median()

Максимальная зарплата 2265604.028722744
Минимальная запрлата 20667.26379327158
Средняя зарплата 167422.30220817294
Медианная зарплата 145017.93753253992


In [43]:
empty_income = len(df[df['total_income'].isna()]) / len(df) 
print('Пропущенных значений {:.0%}'.format(empty_income))

Пропущенных значений 10%


10 % - это не очень много, можно заменить пропущенные значения на медиану для каждой категории.

In [44]:
df['total_income'].isna().sum()

2174

In [46]:
median_by_income_type = df.groupby('income_type').agg({'total_income' : 'median'})
print(median_by_income_type)

                  total_income
income_type                   
безработный      131339.751676
в декрете         53829.130729
госслужащий      150447.935283
компаньон        172357.950966
пенсионер        118514.486412
предприниматель  499163.144947
сотрудник        142594.396847
студент           98201.625314


Заменим пропущенные значения медианой в зависимости от типа занятости:

In [58]:
for element in df.income_type.unique(): 
    df.loc[df['income_type'] == element, 'total_income'] = df.loc[df['income_type'] == element, 'total_income'].fillna(median_by_income_type.loc[element, 'total_income'])

In [59]:
df['total_income'].isna().sum()

0

Пустых строк больше нет.

In [48]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

-1 ребенок - это явно ошибка! 20 детей тоже выглядит очень странно: Давайте посмотрим, у кого такие странные показатели рождаемости:

In [49]:
df[(df['children'] < 0) | (df['children'] == 20)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
705,-1,902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
742,-1,3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
...,...,...,...,...,...,...,...,...,...,...,...,...
21140,-1,1422.668059,44,среднее,1,женат / замужем,0,F,компаньон,0,169562.091999,операции со своей недвижимостью
21325,20,601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,1547.382223,53,среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
21404,20,494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


В данном исследовании нам достаточно знать, что там есть хотя бы один ребенок. 
А вот -1 надо исправлять:

In [50]:
df['children'] = df['children'].abs()

In [51]:
df[df['children'] < 0]['children'].count()

0

Посмотрим, у какого процента человек 20 детей:

In [52]:
twenty_children = len(df[df['children'] == 20]) / len(df) 
print('20 детей у {:.1%} человек'.format(twenty_children))

20 детей у 0.4% человек


0,4% - это, кажется, не очень много. Давайте заменим это значение на 2 - на статистику это не повлияет, т.к. нам достаточно знать, есть ребенок или нет, а это похоже на то, что кто-то случайно нолик поставил - может, хотел скобочку закрыть и не получилось?

In [53]:
df['children'] = df['children'].apply(lambda x: 2 if x == 20 else x)
df[df['children'] == 20].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Отлично! Можем теперь всех пересчитать.

Посмотрим, что еще есть интересного:

In [55]:
df['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Минимальное значение - 0! Что-то тут не так, кажется, младенцам явно нет дела до всяких кредитов (да и как они возвращать их будут - они ж не работают). Посмотрим, сколько таких: 

In [60]:
df[df['dob_years'] == 0].count()

children            101
days_employed       101
dob_years           101
education           101
education_id        101
family_status       101
family_status_id    101
gender              101
income_type         101
debt                101
total_income        101
purpose             101
dtype: int64

101 младенец! Или все-таки нет? Предлагаю заменить на медиану по типу занятости:

In [57]:
df[df['dob_years'] == 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,14439.234121,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,16577.356876,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,15453.312833,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,1574.202821,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
1898,0,15422.689043,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


А на первых строчках debt везде 0, так что есть у них все-таки возможность вернуть кредиты. А я в их годы, эх :)
А серьезно, то 0 лет - это что-то явно неправильное. Давайте что-нибудь с этим придумаем.

In [61]:
zero_years = len(df[df['dob_years'] == 0]) / len(df) 
print('Возрат - 0 лет у {:.1%} человек'.format(zero_years))

Возрат - 0 лет у 0.5% человек


Т.к. таких строк всего 0,5% предлагаю заменить эти значения на медиану:

In [62]:
median_dob_years = df['dob_years'].median()

In [63]:
df['dob_years'] = df['dob_years'].apply(lambda x: median_dob_years if x == 0 else x)

Проверяем:

In [64]:
df[df['dob_years'] == 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Посмотрим на тип занятости:

In [65]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Тут все неплохо, идем дальше:

In [66]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

XNA - это что такое, интересно? Давайте посмотрим:

In [47]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24.0,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Строчка всего лишь одна, думаю, её можно удалить. Замена на медиану тут явно не поможет.

In [67]:
df = df[df['gender'] != 'XNA']

Проверяем:

In [68]:
df['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

Чудненько! Двигаемся дальше.

In [69]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      667
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Приведем все к единому регистру:

In [70]:
df['education'] = df['education'].str.lower()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [71]:
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

Выглядит прекрасно!

In [72]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      623
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

В топе - свадьба и недвижимость! Дальше - разделим их на несколько категорий.

**Вывод** : Убрали пропуски (заменили их на медиану), избавились от отрицательных и неадекватно высоких значений, от странных значений тоже избавились.  Теперь во всех столбцах - порядок! 

### Замена типа данных

Заменим, как и планировали, тип данных в поле days_employed и total_income:

In [73]:
df['days_employed'] = df['days_employed'].astype('int')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [74]:
df['days_employed'].head()

0     8437
1     4024
2     5623
3     4124
4    14177
Name: days_employed, dtype: int64

Заменим также total_income (на статистику копейки не повлияют, но глазу будет приятнее):

In [75]:
df['total_income'] = df['total_income'].astype('int')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [76]:
print(df['total_income'])

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21520    224791
21521    155999
21522     89672
21523    244093
21524     82047
Name: total_income, Length: 21524, dtype: int64


Вывод: Заменили тип данных в двух столбцах: трудовой стаж и доход в месяц. Ни там, ни там сотые и десятые доли не играют особой роли. 
Также нашли ошибку в подсчете трудового стажа у пенсионеров - кажется, там считались часы, а не дни. (исправлено  выше)

### Обработка дубликатов

Т.к. id клиентов в данных нет, сказать наверняка, дубликат это или же совпадение, нельзя. Но по полному совпадению нескольких столбцов можно сделать предположение. 

Посчитаем, сколько дубликатов всех показателей:

In [87]:
df.duplicated().sum()

72

Строки с полным совпадением данных можно считать дубликатами, от которых лучше будет избавиться:

In [89]:
df = df.drop_duplicates().reset_index(drop= True)

Проверяем, есть ли дубликаты теперь:

In [90]:
df.duplicated().sum()

0

Отлично! Избавились от дубликатов, продолжим.

Избавились от полных дубликатов строк - использовался reset_index(), чтобы восстановить индексы строк.

### Лемматизация

In [91]:
from pymystem3 import Mystem
m = Mystem() 

Составим список уникальных целей, на осуществление которых люди хотят взять кредит:

In [92]:
purpose_list = df['purpose'].unique()
print(purpose_list)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [93]:
purpose_list = ' '.join(purpose_list)
print(purpose_list)

покупка жилья приобретение автомобиля дополнительное образование сыграть свадьбу операции с жильем образование на проведение свадьбы покупка жилья для семьи покупка недвижимости покупка коммерческой недвижимости покупка жилой недвижимости строительство собственной недвижимости недвижимость строительство недвижимости на покупку подержанного автомобиля на покупку своего автомобиля операции с коммерческой недвижимостью строительство жилой недвижимости жилье операции со своей недвижимостью автомобили заняться образованием сделка с подержанным автомобилем получение образования автомобиль свадьба получение дополнительного образования покупка своего жилья операции с недвижимостью получение высшего образования свой автомобиль сделка с автомобилем профильное образование высшее образование покупка жилья для сдачи на покупку автомобиля ремонт жилью заняться высшим образованием


Приводим слова к их словарной форме (лемме):

In [94]:
lemmas = m.lemmatize(purpose_list)
print(lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

In [ ]:
list(filter((' ').__ne__, lemmas))
list(filter(lambda a: a != ' ', lemmas))

In [95]:
lemmas.remove('\n')
print(lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

In [99]:
lemmas = list(filter((' ').__ne__, lemmas))

Посчитаем, какие слова чаще всего фигурируют в целях получения кредита. На основе этих данных будем объединять цели в категории

In [100]:
from collections import Counter
print(Counter(lemmas))

Counter({'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


    Можно выделить несколько категорий: недвижимость/жилье, автомобиль, образование, свадьба, строительство. В топе также слово "операция", посмотрим, что за операции: 
    

In [101]:
list = pd.Series(df['purpose'])
# print(list)
print(list[list.str.contains('операц')].unique())

['операции с жильем' 'операции с коммерческой недвижимостью'
 'операции со своей недвижимостью' 'операции с недвижимостью']


Как видно, тут слово операция - это не про врачей и больницы, а про недвижимость. Отдельная категория для операций не нужна.

Првоерим также строительство:

In [102]:
list = pd.Series(df['purpose'])
# print(list)
print(list[list.str.contains('строит')].unique())

['строительство собственной недвижимости' 'строительство недвижимости'
 'строительство жилой недвижимости']


А строительство относится к недвижимости. Посмотрим, что есть с недвижимостью:

In [103]:
list = pd.Series(df['purpose'])
# print(list)
print(list[list.str.contains('недвижим')].unique())

['покупка недвижимости' 'покупка коммерческой недвижимости'
 'покупка жилой недвижимости' 'строительство собственной недвижимости'
 'недвижимость' 'строительство недвижимости'
 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'операции со своей недвижимостью'
 'операции с недвижимостью']


А недвижимость - это не всегда про жилую, поэтому будет 5 категорий: жилье, недвижимость, образование, автомобиль, свадьба.

Посчитали число упоминаний отдельных слов в целях получения кредита, выделали 5 категорий.

### Категоризация данных

Напишем функцию для разделения на категории:

In [104]:
def purpose_category(x):
    if 'жил' in x:
        return 'жилье'
    elif 'авто' in x:
        return 'автомобиль'
    elif 'образов' in x:
        return 'образование'
    elif 'свадьб' in x:
        return 'свадьба'
    elif 'недвижим' in x:
        return 'недвижимость'
    else:
        return 'другое'

Добавим отдельную колонку с категориями:

In [105]:
df['purpose_category'] = df['purpose'].apply(lambda x: purpose_category(x))
df
# df[df['purpose_category'] == 'другое']['purpose_category'].count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21447,1,4529,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилье
21448,0,14330,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21449,1,2113,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21450,3,3112,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


Проверим, к какой категории относится "покупка жилой недвижимости" и "покупка коммерческой недвижимости":

In [106]:
df[df['purpose'] == 'покупка жилой недвижимости']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
14,0,1844,56.0,высшее,0,гражданский брак,1,F,компаньон,1,165127,покупка жилой недвижимости,жилье
96,0,1574,44.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилой недвижимости,жилье
116,1,540,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,83366,покупка жилой недвижимости,жилье
134,1,4171,46.0,неоконченное высшее,2,в разводе,3,F,сотрудник,0,166015,покупка жилой недвижимости,жилье
201,0,6542,39.0,среднее,1,женат / замужем,0,F,компаньон,0,86996,покупка жилой недвижимости,жилье
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21300,0,14632,57.0,среднее,1,женат / замужем,0,F,пенсионер,0,134910,покупка жилой недвижимости,жилье
21318,2,1547,53.0,среднее,1,женат / замужем,0,M,компаньон,0,172357,покупка жилой недвижимости,жилье
21347,1,108,40.0,среднее,1,женат / замужем,0,M,сотрудник,1,155849,покупка жилой недвижимости,жилье
21375,0,96,36.0,среднее,1,гражданский брак,1,F,компаньон,0,276254,покупка жилой недвижимости,жилье


In [107]:
df[df['purpose'] == 'покупка коммерческой недвижимости']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
11,0,792,40.0,среднее,1,женат / замужем,0,F,сотрудник,0,77069,покупка коммерческой недвижимости,недвижимость
70,1,2802,28.0,высшее,0,женат / замужем,0,M,госслужащий,0,207561,покупка коммерческой недвижимости,недвижимость
119,0,15924,60.0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,31534,покупка коммерческой недвижимости,недвижимость
140,0,3017,24.0,среднее,1,гражданский брак,1,F,сотрудник,0,102110,покупка коммерческой недвижимости,недвижимость
173,0,9099,49.0,среднее,1,женат / замужем,0,M,компаньон,0,416461,покупка коммерческой недвижимости,недвижимость
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21258,1,1375,23.0,среднее,1,Не женат / не замужем,4,F,сотрудник,0,93200,покупка коммерческой недвижимости,недвижимость
21336,0,9416,48.0,среднее,1,Не женат / не замужем,4,F,сотрудник,0,219716,покупка коммерческой недвижимости,недвижимость
21365,0,5435,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,173366,покупка коммерческой недвижимости,недвижимость
21384,0,448,49.0,среднее,1,женат / замужем,0,F,сотрудник,1,156320,покупка коммерческой недвижимости,недвижимость


Цели поделены на категории

Добавим столбец есть дети/нет детей:

In [108]:
df['have_children'] = df['children'].apply(lambda x: 1 if x > 0 else 0)

Проверяем:

In [109]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,have_children
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,1
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,0
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,1
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21447,1,4529,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилье,1
21448,0,14330,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,0
21449,1,2113,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,1
21450,3,3112,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,1


Теперь займемся доходом:

In [112]:
df.groupby('income_type')['total_income'].median()

income_type
безработный        131339
в декрете           53829
госслужащий        150447
компаньон          172357
пенсионер          118514
предприниматель    499163
сотрудник          142594
студент             98201
Name: total_income, dtype: int64

Посмотрели на медиану дохода по типу занятости, разделим всех на такие группы: 0 - 100 тысяч, 100 тысяч - 145 тысяч, 145 тыс - 200 тысяч и больше 200 тысяч: (т.е. в первую группу вероятнее всего попадут люди в декрете и студенты, во вторую - безработные, пенсионеры и сотрудники, в третью - госслужащие и компаньоны, и в четвертую - предприниматели)

In [113]:
def income_type_group(x):
    if x <= 100000:
        return "0 - 100 тысяч"
    elif 100000 < x <= 145000:
        return "100 - 145 тысяч"
    elif 145000 < x <= 200000:
        return "145 - 200 тысяч"
    else: 
        return "больше 200 тысяч"

In [116]:
df['total_income_group'] = df['total_income'].apply(income_type_group)

In [117]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,have_children,total_income_group
0,1,8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,1,больше 200 тысяч
1,1,4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1,100 - 145 тысяч
2,0,5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,0,145 - 200 тысяч
3,3,4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,1,больше 200 тысяч
4,0,14177,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0,145 - 200 тысяч
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21447,1,4529,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,жилье,1,больше 200 тысяч
21448,0,14330,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,0,145 - 200 тысяч
21449,1,2113,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,1,0 - 100 тысяч
21450,3,3112,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,1,больше 200 тысяч


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [118]:
have_children_debt = df[['have_children', 'debt']][df['have_children'] == 1]
havent_children_debt = df[['have_children', 'debt']][df['have_children'] == 0]

have_children_have_debt = have_children_debt['debt'].sum()/len(have_children_debt)
havent_children_have_debt = havent_children_debt['debt'].sum()/len(havent_children_debt)
print('Процент задолженности у людей с детьми: {:.1%}'.format(have_children_have_debt))
print('Процент задолженности у людей без детей: {:.1%}'.format(havent_children_have_debt))

Процент задолженности у людей с детьми: 9.2%
Процент задолженности у людей без детей: 7.5%


In [120]:
# 100*df.groupby('have_children')['debt'].mean().round(3)

**Вывод**: Казалось бы, люди с детьми должны быть более ответственными, но статистика говорит об обратном: бездетные отдают кредиты чаще.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [121]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Так же, как и в предыдущем случае - соберем таблицу из двух колонок:

In [122]:
debt_family_status = df[['debt', 'family_status']]
print(debt_family_status)

       debt     family_status
0         0   женат / замужем
1         0   женат / замужем
2         0   женат / замужем
3         0   женат / замужем
4         0  гражданский брак
...     ...               ...
21447     0  гражданский брак
21448     0   женат / замужем
21449     1  гражданский брак
21450     1   женат / замужем
21451     0   женат / замужем

[21452 rows x 2 columns]


Интересно посмотреть про то, как наличие отношений влияет на возврат кредита:

In [123]:
def relationship_status(x):
    if (x == 'женат / замужем') or (x == 'гражданский брак'):
        return 1
    else:
        return 0
    
debt_family_status['in_relationship'] = debt_family_status['family_status'].apply(lambda x: relationship_status(x))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [124]:
print(debt_family_status.head(20))

    debt     family_status  in_relationship
0      0   женат / замужем                1
1      0   женат / замужем                1
2      0   женат / замужем                1
3      0   женат / замужем                1
4      0  гражданский брак                1
5      0  гражданский брак                1
6      0   женат / замужем                1
7      0   женат / замужем                1
8      0  гражданский брак                1
9      0   женат / замужем                1
10     0   женат / замужем                1
11     0   женат / замужем                1
12     0  гражданский брак                1
13     0   женат / замужем                1
14     1  гражданский брак                1
15     0   женат / замужем                1
16     0   женат / замужем                1
17     0  гражданский брак                1
18     0    вдовец / вдова                0
19     0         в разводе                0


Теперь можно посмотреть, влияет ли семейное положение на возврат кредита:

In [125]:
in_relationship = debt_family_status[['in_relationship', 'debt']][debt_family_status['in_relationship'] == 1]
not_in_relationship = debt_family_status[['in_relationship', 'debt']][debt_family_status['in_relationship'] == 0]

in_relationship_have_debt = in_relationship['debt'].sum()/len(in_relationship)
not_in_relationship_have_debt = not_in_relationship['debt'].sum()/len(not_in_relationship)
print('Процент задолженности у людей в отношениях: {:.1%}'.format(in_relationship_have_debt))
print('Процент задолженности у людей без отношений: {:.1%}'.format(not_in_relationship_have_debt))

Процент задолженности у людей в отношениях: 8.0%
Процент задолженности у людей без отношений: 8.5%


Теперь посчитаем то же самое для каждого семейного статуса:

In [126]:
data_pivot_family_status = df.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
# print(data_pivot_family_status)
data_pivot_family_status['sum'] = data_pivot_family_status[1] + data_pivot_family_status[0]
data_pivot_family_status['Процент задолженности %'] = round(data_pivot_family_status[1] / data_pivot_family_status['sum'], 3) * 100

print(data_pivot_family_status)




debt                       0    1    sum  Процент задолженности %
family_status                                                    
Не женат / не замужем   2536  274   2810                      9.8
в разводе               1110   85   1195                      7.1
вдовец / вдова           896   63    959                      6.6
гражданский брак        3761  388   4149                      9.4
женат / замужем        11408  931  12339                      7.5


**Вывод**: Чаще всего кредиты не отдают неженатые и незамужние, а реже - вдовы и вдовцы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [127]:
debt_by_total_income = df[['debt','total_income', 'total_income_group']]

In [131]:
print(debt_by_total_income)

       debt  total_income total_income_group
0         0        253875   больше 200 тысяч
1         0        112080    100 - 145 тысяч
2         0        145885    145 - 200 тысяч
3         0        267628   больше 200 тысяч
4         0        158616    145 - 200 тысяч
...     ...           ...                ...
21447     0        224791   больше 200 тысяч
21448     0        155999    145 - 200 тысяч
21449     1         89672      0 - 100 тысяч
21450     1        244093   больше 200 тысяч
21451     0         82047      0 - 100 тысяч

[21452 rows x 3 columns]


In [132]:
df_pivot_total_income = debt_by_total_income.pivot_table(index='total_income_group', columns='debt', values='total_income', aggfunc='count')
print(df_pivot_total_income)

debt                   0    1
total_income_group           
0 - 100 тысяч       4109  354
100 - 145 тысяч     6083  583
145 - 200 тысяч     4811  446
больше 200 тысяч    4708  358


In [133]:
df_pivot_total_income['sum'] = df_pivot_total_income[1] + df_pivot_total_income[0]
df_pivot_total_income['Процент задолженности %'] = round(df_pivot_total_income[1] / df_pivot_total_income['sum'], 3) * 100

print(df_pivot_total_income)

debt                   0    1   sum  Процент задолженности %
total_income_group                                          
0 - 100 тысяч       4109  354  4463                      7.9
100 - 145 тысяч     6083  583  6666                      8.7
145 - 200 тысяч     4811  446  5257                      8.5
больше 200 тысяч    4708  358  5066                      7.1


**Вывод**: Выходит, что кредит чаще возвращают люди с доходом меньше 100 тысяч и выше 200 тысяч.

- Как разные цели кредита влияют на его возврат в срок?

In [134]:
df_pivot_purposes = df.pivot_table(index='purpose_category', columns='debt', values='purpose', aggfunc='count')
print(df_pivot_purposes)

debt                 0    1
purpose_category           
автомобиль        3903  403
жилье             5293  397
недвижимость      4735  385
образование       3643  370
свадьба           2137  186


In [135]:
df_pivot_purposes['sum'] = df_pivot_purposes[1] + df_pivot_purposes[0]
df_pivot_purposes['Процент задолженности %'] = round(df_pivot_purposes[1] / df_pivot_purposes['sum'], 3) * 100

print(df_pivot_purposes)

debt                 0    1   sum  Процент задолженности %
purpose_category                                          
автомобиль        3903  403  4306                      9.4
жилье             5293  397  5690                      7.0
недвижимость      4735  385  5120                      7.5
образование       3643  370  4013                      9.2
свадьба           2137  186  2323                      8.0


Реже всего люди выплачивают кредиты, взятые на автомобили и на образование. Чаще возвращают кредиты, взятые на сделки с жильем и недвижимостью.

## Шаг 4. Общий вывод

Идеальный заемщик - бездетный, разведенный человек с доходом больше 200 тысяч, берущий кредит на жилье.

Плохой заемщик - человек с детьми, не женат или не замужем, имеющий зарплату 100 - 145 тысяч и берущий кредит на автомобиль. 